In [14]:
%pip install praw
%pip install openai==0.28


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.94.0
    Uninstalling openai-1.94.0:
      Successfully uninstalled openai-1.94.0


In [1]:
# Required libraries
import re
import praw
import openai
import textwrap
import prawcore
from IPython.display import display, Markdown

In [ ]:
#Credentials of reddit and open ai api
REDDIT_CLIENT_ID = '5pLuHrA8wEe-RJJuO1Tnpw'
REDDIT_CLIENT_SECRET = 'Urg0pt-AOXOqt0gboZt5B3-ueKIhGQ'
REDDIT_USER_AGENT = 'PersonaGenerator/1.0 by Competitive-Candy257'
OPENAI_API_KEY = '' # Set your api here

In [ ]:
def validate_credentials():

    """API credentials are stored in environment variables"""
    if (REDDIT_CLIENT_ID == 'YOUR_REDDIT_CLIENT_ID' or
        REDDIT_CLIENT_SECRET == 'YOUR_REDDIT_CLIENT_SECRET' or
        'YOUR_REDDIT_USERNAME' in REDDIT_USER_AGENT):
        raise ValueError("Please replace with your actual API keys")

    # Initialize APIs, if credentials valid
    try:
        reddit = praw.Reddit(
            client_id=REDDIT_CLIENT_ID,
            client_secret=REDDIT_CLIENT_SECRET,
            user_agent=REDDIT_USER_AGENT
        )
        # Test authentication of API
        reddit.user.me()
        return reddit
    except prawcore.exceptions.ResponseException as e:
        if e.response.status_code == 401:
            raise ValueError("Invalid Reddit API credentials")
        raise
    except Exception as e:
        raise RuntimeError(f"Reddit API initialization failed: {str(e)}")


In [ ]:
# Initialize Reddit API with validation
try:
    reddit = validate_credentials()
    print("Reddit API validated successfully")
except Exception as e:
    print(f"Credential Error: {str(e)}")
    reddit = None

Reddit API credentials validated successfully


In [ ]:
# Initialize OpenAI
openai.api_key = OPENAI_API_KEY

def validate_reddit_url(url):
    """Reddit profile URL"""
    pattern = r'https?://www\.reddit\.com/user/([^/]+)/?'
    match = re.search(pattern, url)
    if not match:
        raise ValueError("Invalid Reddit profile URL format.")
    return match.group(1)


In [ ]:
# Scrape Redditor's comments and submissions with error handling
def scrape_redditor_content(username):

    if not reddit:
        raise RuntimeError("Reddit API not initialized due to credential issues")

    try:
        user = reddit.redditor(username)

        comments = []
        try:
            for comment in user.comments.new(limit=100):
                comments.append({
                    'type': 'comment',
                    'content': comment.body,
                    'subreddit': comment.subreddit.display_name,
                    'created_utc': comment.created_utc,
                    'url': f'https://reddit.com{comment.permalink}',
                    'score': comment.score
                })
        except prawcore.exceptions.Forbidden:
            print(f"Comments are private for u/{username}")

        submissions = []
        try:
            for submission in user.submissions.new(limit=50):
                submissions.append({
                    'type': 'submission',
                    'title': submission.title,
                    'content': submission.selftext,
                    'subreddit': submission.subreddit.display_name,
                    'created_utc': submission.created_utc,
                    'url': f'https://reddit.com{submission.permalink}',
                    'score': submission.score
                })
        except prawcore.exceptions.Forbidden:
            print(f"Submissions are private for u/{username}")

        if not comments and not submissions:
            raise ValueError(f"No public content available for u/{username} or profile doesn't exist")

        return comments + submissions
    except prawcore.exceptions.NotFound:
        raise ValueError(f"User u/{username} not found")
    except prawcore.exceptions.ResponseException as e:
        if e.response.status_code == 401:
            raise RuntimeError("Reddit API credentials expired ")
        raise RuntimeError(f"Reddit API error: {str(e)}")
    except Exception as e:
        raise RuntimeError(f"Error scraping data: {str(e)}")

In [ ]:
# Generate user persona using GPT-4
def generate_persona(content_data):
    
    if not content_data:
        raise ValueError("No content available for persona generation")

    content_chunks = []
    for idx, item in enumerate(content_data[:30]):
        source_ref = f"[Source {idx+1}]"
        if item['type'] == 'comment':
            text = f"{source_ref} COMMENT in r/{item['subreddit']}: {item['content']}"
        else:
            text = f"{source_ref} POST: {item['title']}\n{item['content']}"
        content_chunks.append(text)

    content_str = "\n\n".join(content_chunks)[:12000]

    system_msg = textwrap.dedent("""
    We are Creating user personas from Reddit data.
    Generate a comprehensive persona in EXACTLY the following format:

    # [Name]

    **AGE** | [Value]
    **OCCUPATION** | [Value]
    **STATUS** | [Value]
    **LOCATION** | [Value]
    **TIER** | [Value]
    **ARCHETYPE** | [Value]

    ---

    ## [4-5 Word Trait Summary]

    ### MOTIVATIONS

    - **[Main Motivation Category]**
      [Sub-motivation 1]
      [Sub-motivation 2]
      [Sub-motivation 3]

    ### PERSONALITY

    | [Trait Dimension 1] | [Trait Dimension 2] |
    |---------------------|---------------------|
    | [Value]             | [Value]             |
    | [Value]             | [Value]             |
    | [Value]             | [Value]             |
    | [Value]             | [Value]             |

    **BEHAVIOUR & HABITS**
    - [Behavior point 1] [Source X]
    - [Behavior point 2] [Source Y]

    **GOALS & NEEDS**
    - [Goal 1] [Source X]
    - [Goal 2] [Source Y]

    **FRUSTRATIONS**
    - [Frustration 1] [Source X]
    - [Frustration 2] [Source Y]

    "[Direct Quote]" [Source X]

    RULES:
    1. Use name '______' if unavailable
    2. STRUCTURE MUST MATCH EXACTLY
    3. Use MBTI dimensions: Introvert/Extrovert, Intuition/Sensing, Feeling/Thinking, Perceiving/Judging
    4. Tier must be: Early Adoption, Mainstream, or Late Adoption
    5. Archetype must be standard persona type (e.g., The Creator)
    6. Motivation categories: CONVENIENCE, HEALTH, SOCIAL, etc.
    7. Trait summary = 4-5 adjective phrase (e.g., Practical Appetable Spontaneous Active)
    8. Cite sources after EACH point/quote
    9. NEVER invent information
    """
    )



    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": f"Redditor content:\n\n{content_str}"}
            ],
            temperature=0.3,
            max_tokens=1500
        )
        return response.choices[0].message['content']
    except Exception as e:
        raise Exception(f"OpenAI API error: {str(e)}")

In [25]:
# Save persona with source references
def save_persona(username, persona, content_data):

    source_index = "\n\nSOURCE REFERENCES:\n"
    for idx, item in enumerate(content_data[:30]):
        source_index += f"[Source {idx+1}] {item['url']}\n"

    filename = f"{username}_persona.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(persona)
        f.write(source_index)

    return filename

In [26]:
# Main function to generate persona from Reddit URL
def generate_persona_from_reddit(url):

    if not reddit:
        return "reddit API are invalid"

    try:
        # Validate and scrape
        username = validate_reddit_url(url)
        print(f"Scraping data for u/{username}.")
        content_data = scrape_redditor_content(username)

        # persona Generate
        print("Generating persona with GPT-4.")
        persona = generate_persona(content_data)

        # output save
        filename = save_persona(username, persona, content_data)
        print(f"Persona saved to {filename}.")

        # Display in Jupyter
        display(Markdown(persona))
        print(f"\nFull persona with sources saved to {filename}")

        return persona

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(error_msg)
        return error_msg

In [27]:
generate_persona_from_reddit("https://www.reddit.com/user/Hungry-Move-6603/")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Scraping data for u/Hungry-Move-6603.
Generating persona with GPT-4.
Persona saved to Hungry-Move-6603_persona.txt.


# ______

**AGE** | 25-35
**OCCUPATION** | Business Owner
**STATUS** | Single
**LOCATION** | Lucknow, India
**TIER** | Early Adoption
**ARCHETYPE** | The Explorer

---

## Health Conscious, Socially Aware, Observant

### MOTIVATIONS

- **CONVENIENCE**
  Prefers quick and easy meals
  Seeks efficient solutions to problems
  Values practicality in daily life

- **SOCIAL**
  Engages in community discussions
  Observes and comments on societal issues
  Expresses opinions on public matters

### PERSONALITY

| Introvert/Extrovert | Intuition/Sensing |
|---------------------|---------------------|
| Introvert             | Sensing             |
| Prefers online interactions | Observant of surroundings |
| Enjoys solitary activities like reading | Practical and realistic |

**BEHAVIOUR & HABITS**
- Frequently comments on societal and local issues [Source 1,2,3,4,6,7,8,9,14]
- Shows interest in healthy and quick meals [Source 5,10]

**GOALS & NEEDS**
- Seeks to maintain a healthy lifestyle [Source 5,10]
- Desires productive activities and social engagement [Source 12,13]

**FRUSTRATIONS**
- Frustrated with corruption and societal issues [Source 2,3,9]
- Displeased with the quality of food options available [Source 10]

"Never seen anything of this sort in capital city of India. So yes I did not expect 🤪" [Source 8]


Full persona with sources saved to Hungry-Move-6603_persona.txt


'# ______\n\n**AGE** | 25-35\n**OCCUPATION** | Business Owner\n**STATUS** | Single\n**LOCATION** | Lucknow, India\n**TIER** | Early Adoption\n**ARCHETYPE** | The Explorer\n\n---\n\n## Health Conscious, Socially Aware, Observant\n\n### MOTIVATIONS\n\n- **CONVENIENCE**\n  Prefers quick and easy meals\n  Seeks efficient solutions to problems\n  Values practicality in daily life\n\n- **SOCIAL**\n  Engages in community discussions\n  Observes and comments on societal issues\n  Expresses opinions on public matters\n\n### PERSONALITY\n\n| Introvert/Extrovert | Intuition/Sensing |\n|---------------------|---------------------|\n| Introvert             | Sensing             |\n| Prefers online interactions | Observant of surroundings |\n| Enjoys solitary activities like reading | Practical and realistic |\n\n**BEHAVIOUR & HABITS**\n- Frequently comments on societal and local issues [Source 1,2,3,4,6,7,8,9,14]\n- Shows interest in healthy and quick meals [Source 5,10]\n\n**GOALS & NEEDS**\n- See

In [28]:
generate_persona_from_reddit("https://www.reddit.com/user/kojied/")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Scraping data for u/kojied.


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Generating persona with GPT-4.
Persona saved to kojied_persona.txt.


# ______

**AGE** | 30-35
**OCCUPATION** | Game Developer
**STATUS** | Single
**LOCATION** | New York City, USA
**TIER** | Early Adoption
**ARCHETYPE** | The Strategist

---

## Strategic, Tech-Savvy, Gaming Enthusiast

### MOTIVATIONS

- **CONVENIENCE**
  Efficient gaming strategies
  Utilizing technology for convenience
  Streamlining processes

### PERSONALITY

| Introvert/Extrovert | Intuition/Sensing |
|---------------------|---------------------|
| Introvert             | Intuition             |
| Analytical            | Detail-oriented     |
| Reflective            | Observant            |
| Independent          | Innovative           |

**BEHAVIOUR & HABITS**
- Frequently engages in gaming discussions, particularly strategy games like ManorLords and Civ5 [Source 1, 2, 4, 6, 7, 8, 9, 10, 20, 24, 25, 29]
- Shows interest in technology and AI, particularly in the context of gaming and digital art [Source 3, 18, 19, 21, 26]

**GOALS & NEEDS**
- Seeks to optimize gaming strategies for efficiency and success [Source 1, 7, 10, 20, 24, 29]
- Desires to learn and implement new technology for personal and professional growth [Source 3, 18, 19, 21]

**FRUSTRATIONS**
- Disappointed with the lack of late game content in some games [Source 4]
- Frustrated with the decline of interest and innovation in the NFT space [Source 12]

"Millenials have less kids, so they have more time and money to spend on themselves. Millenials are also getting married less and later in life, meaning we allocate more effort on self care" [Source 13]


Full persona with sources saved to kojied_persona.txt


'# ______\n\n**AGE** | 30-35\n**OCCUPATION** | Game Developer\n**STATUS** | Single\n**LOCATION** | New York City, USA\n**TIER** | Early Adoption\n**ARCHETYPE** | The Strategist\n\n---\n\n## Strategic, Tech-Savvy, Gaming Enthusiast\n\n### MOTIVATIONS\n\n- **CONVENIENCE**\n  Efficient gaming strategies\n  Utilizing technology for convenience\n  Streamlining processes\n\n### PERSONALITY\n\n| Introvert/Extrovert | Intuition/Sensing |\n|---------------------|---------------------|\n| Introvert             | Intuition             |\n| Analytical            | Detail-oriented     |\n| Reflective            | Observant            |\n| Independent          | Innovative           |\n\n**BEHAVIOUR & HABITS**\n- Frequently engages in gaming discussions, particularly strategy games like ManorLords and Civ5 [Source 1, 2, 4, 6, 7, 8, 9, 10, 20, 24, 25, 29]\n- Shows interest in technology and AI, particularly in the context of gaming and digital art [Source 3, 18, 19, 21, 26]\n\n**GOALS & NEEDS**\n- Se